In [10]:
import bw_processing as bwp
import matrix_utils as mu
import bw2calc as bc
import numpy as np
import pandas as pd

In [7]:
#!pip install matrix-utils

In [8]:
#!pip install bw-processing

In [ ]:
def get_elec_mix_data_for_Scn_x_Year_t(user_scn, user_yr, user_dt):
    """ 
    input: user-defined elec_mix for user_scn, user_yr 
    output:
        get_elec_mix_data: to be used in scenario_vector
            the np.array for the elec_mix for user_scn/user_yr: e.g., 
            [[stores_coal_array],[stores_wind_array],[stores_gas_array]...] 
    """ 
    get_elec_mix_data = np.array( )
    return (get_elec_mix_data)

In [ ]:
def get_elec_A_mat_indice_for_Scn_x_Year_t(user_scn, user_yr, elec_act):
    """ 
    input: elec_act: the bw2 activity 
    output:
        get_indice_from_A: indice for the elec_act from the A matrix
    """ 
    lca = bc.LCA(demand = {elec_act:1})
    lca.lci()
    #BW2 
    get_indice_from_A = lca.activity_dict[elec_act]
    #BW25 
    return (get_indice_from_A)

In [14]:
def get_elec_A_array_for_Scn_x_Year_t(user_scn, user_yr, get_indice_from_A, get_elec_mix_data): 
    """ 
    get_indice_from_A: the col_indice from A
    user_scn : the user-defined scenario for the new elec_mix
    user_yr: the year for the new elec_mix under user_scn 
    get_elec_mix_data: 
        the np.array for the elec_mix for user_scn/user_yr: e.g., 
        [[stores_coal_array],[stores_wind_array],[stores_gas_array]...]
    """ 
    elec_scn = bwp.create_datapackage() 
    elec_scn.add_persistent_array(
        matrix='technosphere_matrix',
        indices_array=np.array([
                get_indice_from_A, # new elec_mix indice for scn / Year
            ], 
            dtype=bwp.INDICES_DTYPE
        ),
        # the array shape should be (N_of_elec_tech, )
        data_array = get_elec_mix_data.reshape((1, -1)), 
        flip_array = np.array([True]),
        name='scenario_' + str(user_scn) + 'year' + str(user_yr)
    )
    return elec_scn

In [ ]:
def get_elec_lcia_score(dp_A, get_indice_from_A, elec_scn, impact_method): 
    """ 
    dp_A: the original / or new_elec_mix A_matrix  
    get_indice_from_A:  
    elec_scn: 
    impact_method: the impact methods to run
    """ 
    lca = bc.LCA(
        demand={get_indice_from_A: 1},
        data_objs=[dp_A, elec_scn],
        method = impact_method, 
        use_arrays=True,
    )
    lca.lci()
    lca.lcia()